In [8]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import dict_map as dm

filepath = './models/train300wWeights'
# Load the model
model = load_model(filepath, compile = True)

In [9]:
df = pd.read_csv('Daten/trainingdata300.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)
# df = df.head(1000)
token2idx, idx2token = dm.get_dict_map(df, 'token')
tag2idx, idx2tag = dm.get_dict_map(df, 'tag')

df['Word_idx'] = df['Wort'].map(token2idx)
df['Tag_idx'] = df['Attribut'].map(tag2idx)

df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
#df_group
import pad_train_test as ptt
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = ptt.get_pad_train_test_val(df_group, df, tag2idx)

<ipython-input-9-ffa0064ed6ee>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
train_tokens length: 26331 
train_tags length: 26331 
test_tokens length: 3902 
test_tags: 3902 
val_tokens: 8778 
val_tags: 8778


In [10]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test_tokens, np.array(test_tags), batch_size=512)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(test_tokens[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
8/8 [==============================] - 8s 862ms/step - loss: 7.0737 - accuracy: 0.0057
test loss, test acc: [7.073709011077881, 0.005673158913850784]
Generate predictions for 3 samples
predictions shape: (3, 300, 8)


In [4]:
predictions

array([[[0.03265458, 0.05867001, 0.07293681, ..., 0.03679058,
         0.19502704, 0.06096945],
        [0.00733751, 0.02066729, 0.03435301, ..., 0.00869032,
         0.16527705, 0.02474638],
        [0.00328501, 0.01201597, 0.02356027, ..., 0.00378194,
         0.12559886, 0.01505491],
        ...,
        [0.00073424, 0.00290163, 0.00664687, ..., 0.00054984,
         0.0095855 , 0.00148857],
        [0.00077047, 0.00299882, 0.00689066, ..., 0.00057579,
         0.00973489, 0.00154147],
        [0.00080558, 0.00309201, 0.00711741, ..., 0.0006005 ,
         0.0098879 , 0.0015928 ]],

       [[0.03229791, 0.05848663, 0.07279234, ..., 0.03642419,
         0.19276194, 0.06028431],
        [0.00724463, 0.02064633, 0.03444205, ..., 0.00860842,
         0.16094261, 0.02436695],
        [0.00325559, 0.01207753, 0.02366722, ..., 0.00375996,
         0.12165047, 0.01485828],
        ...,
        [0.00073424, 0.00290163, 0.00664687, ..., 0.00054984,
         0.0095855 , 0.00148857],
        [0.0

In [5]:
classes = np.argmax(predictions, axis = 1)

In [6]:
# print(predictions)
print(classes)

[[  0   0   0   0 103   0   0   0]
 [  0   0   0   0 215   0   0   0]
 [  0   0   0   0 208   0   0   0]]


In [13]:
df.groupby('Attribut').count()

,satzId,Wort,Word_idx,Tag_idx
Attribut,,,,
B-Brand,67464,67464,67464,67464
B-Modelnumber,33296,33115,33296,33296
E-Brand,4521,4521,4521,4521
E-Modelnumber,7143,7143,7143,7143
I-Brand,1068,1068,1068,1068
I-Modelnumber,7123,7123,7123,7123
O,4807866,4807703,4807866,4807866


In [7]:
df.isna().sum()

satzId        0
Wort        344
Attribut      3
Word_idx      0
Tag_idx       0
dtype: int64